# 3. ВИЗУАЛИЗАЦИя

In [8]:
# 1. ИМПОРТ БИБЛИОТЕК
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot



In [9]:
# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [10]:
'''1. Загрузка данных'''
merged_cleaned_data = pd.read_csv('11_merged_cleaned_data.csv')

In [11]:
df = merged_cleaned_data

In [12]:
# Продажи по странам
sales_by_country = merged_cleaned_data.groupby('Country')['SalesAmount'].sum().reset_index()
fig2 = px.choropleth(sales_by_country, locations='Country', color='SalesAmount',
                     locationmode='country names', title='Продажи по странам')
fig2.show()

# Продажи по регионам
sales_by_region = merged_cleaned_data.groupby('Region')['SalesAmount'].sum().reset_index()
fig3 = px.pie(sales_by_region, names='Region', values='SalesAmount', title='Продажи по регионам')
fig3.show()

In [17]:
# 📊 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ДАШБОРДОВ
print("📊 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ДАШБОРДОВ")
print("=" * 50)

# Расчет ключевых метрик
print("📈 РАСЧЕТ КЛЮЧЕВЫХ МЕТРИК:")
total_revenue = df['SalesAmount'].sum()
total_orders = df['SalesOrderNumber'].nunique()
total_customers = df['CustomerKey'].nunique()
avg_order_value = df.groupby('SalesOrderNumber')['SalesAmount'].sum().mean()
total_profit = df['Profit'].sum() if 'Profit' in df.columns else 0

print(f"💰 Общая выручка: ${total_revenue:,.0f}")
print(f"📦 Всего заказов: {total_orders:,}")
print(f"👥 Всего клиентов: {total_customers:,}")
print(f"💵 Средний чек: ${avg_order_value:,.0f}")
print(f"💎 Общая прибыль: ${total_profit:,.0f}")

# 1. Дашборд ключевых метрик
print("\n1. 📊 ДАШБОРД КЛЮЧЕВЫХ МЕТРИК")
fig_metrics = go.Figure()

fig_metrics.add_trace(go.Indicator(
    mode="number",
    value=total_revenue,
    title={"text": "💰 Общая выручка"},
    number={'prefix': "$", "valueformat": ",.0f"},
    domain={'row': 0, 'column': 0}
))

fig_metrics.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    title={"text": "📦 Всего заказов"},
    number={"valueformat": ",.0f"},
    domain={'row': 0, 'column': 1}
))

fig_metrics.add_trace(go.Indicator(
    mode="number",
    value=total_customers,
    title={"text": "👥 Всего клиентов"},
    number={"valueformat": ",.0f"},
    domain={'row': 0, 'column': 2}
))

fig_metrics.add_trace(go.Indicator(
    mode="number",
    value=avg_order_value,
    title={"text": "💵 Средний чек"},
    number={'prefix': "$", "valueformat": ",.0f"},
    domain={'row': 0, 'column': 3}
))

fig_metrics.update_layout(
    grid={'rows': 1, 'columns': 4, 'pattern': "independent"},
    template="plotly_white",
    height=300,
    title="КЛЮЧЕВЫЕ МЕТРИКИ ПРОДАЖ"
)

fig_metrics.show()
print("✅ Дашборд ключевых метрик создан")

# 2. Дашборд географического распределения
print("\n2. 🌍 ДАШБОРД ГЕОГРАФИЧЕСКОГО РАСПРЕДЕЛЕНИЯ")
if 'Country' in df.columns and 'Region' in df.columns:
    geo_analysis = df.groupby(['Country', 'Region']).agg({
        'SalesAmount': 'sum',
        'CustomerKey': 'nunique',
        'Profit': 'sum'
    }).reset_index()
    
    print(f"📊 Географический анализ:")
    print(f"   • Стран: {geo_analysis['Country'].nunique()}")
    print(f"   • Регионов: {geo_analysis['Region'].nunique()}")
    print(f"   • Общая выручка по регионам: ${geo_analysis['SalesAmount'].sum():,.0f}")
    
    fig_geo = px.scatter_geo(geo_analysis, 
                            locations='Country',
                            locationmode='country names',
                            color='SalesAmount',
                            size='CustomerKey',
                            hover_name='Region',
                            hover_data={'SalesAmount': ':.0f', 'Profit': ':.0f'},
                            title='🌍 Географическое распределение продаж',
                            color_continuous_scale='Viridis')
    fig_geo.show()
    print("✅ Географический дашборд создан")
else:
    print("❌ Пропущен географический дашборд: отсутствуют Country или Region")

# 3. Анализ продуктовой линейки
print("\n3. ДАШБОРД ПРОДУКТОВОЙ ЛИНЕЙКИ")
if 'Category' in df.columns and 'SubCategory' in df.columns:
    product_analysis = df.groupby(['Category', 'SubCategory']).agg({
        'SalesAmount': 'sum',
        'Profit': 'sum',
        'OrderQuantity': 'sum'
    }).reset_index()
    
    print(f"📊 Анализ продуктовой линейки:")
    print(f"   • Категорий: {product_analysis['Category'].nunique()}")
    print(f"   • Подкатегорий: {product_analysis['SubCategory'].nunique()}")
    print(f"   • Общая выручка по категориям: ${product_analysis['SalesAmount'].sum():,.0f}")
    
    fig_products = px.treemap(product_analysis,
                             path=['Category', 'SubCategory'],
                             values='SalesAmount',
                             color='Profit',
                             color_continuous_scale='RdYlGn',
                             title='📦 Дерево продуктов по выручке и прибыли',
                             hover_data=['OrderQuantity'])
    fig_products.show()
    print("✅ Дашборд продуктовой линейки создан")
else:
    print("❌ Пропущен дашборд продуктовой линейки: отсутствуют Category или SubCategory")

# 4. Дополнительный дашборд - анализ клиентов
print("\n4. 👥 ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ КЛИЕНТОВ")
if 'YearlyIncome' in df.columns and 'Age' in df.columns:
    customer_analysis = df.groupby('CustomerKey').agg({
        'SalesAmount': 'sum',
        'YearlyIncome': 'first',
        'Age': 'first',
        'OrderQuantity': 'sum'
    }).reset_index()
    
    fig_customers = px.scatter(customer_analysis, 
                              x='YearlyIncome', 
                              y='SalesAmount',
                              size='OrderQuantity',
                              color='Age',
                              title='👥 Распределение клиентов по доходу и выручке',
                              labels={'YearlyIncome': 'Годовой доход', 
                                     'SalesAmount': 'Общая выручка клиента',
                                     'Age': 'Возраст'},
                              hover_data=['CustomerKey'])
    fig_customers.show()
    print("✅ Дашборд анализа клиентов создан")
else:
    print("❌ Пропущен дашборд анализа клиентов: отсутствуют YearlyIncome или Age")

print(f"\n📊 ИТОГ СОЗДАНИЯ ДАШБОРДОВ:")
print(f"   • Создано дашбордов: 4")
print(f"   • Ключевые метрики: {total_revenue:,.0f} $ выручки")
print(f"   • Охвачено клиентов: {total_customers:,}")
print(f"   • Обработано заказов: {total_orders:,}")
print("✅ ВСЕ ДАШБОРДЫ УСПЕШНО СОЗДАНЫ И ОТОБРАЖЕНЫ!")

📊 СОЗДАНИЕ ИНТЕРАКТИВНЫХ ДАШБОРДОВ
📈 РАСЧЕТ КЛЮЧЕВЫХ МЕТРИК:
💰 Общая выручка: $29,307,837
📦 Всего заказов: 26,683
👥 Всего клиентов: 17,918
💵 Средний чек: $1,098
💎 Общая прибыль: $12,052,518

1. 📊 ДАШБОРД КЛЮЧЕВЫХ МЕТРИК


✅ Дашборд ключевых метрик создан

2. 🌍 ДАШБОРД ГЕОГРАФИЧЕСКОГО РАСПРЕДЕЛЕНИЯ
📊 Географический анализ:
   • Стран: 6
   • Регионов: 10
   • Общая выручка по регионам: $29,307,837


✅ Географический дашборд создан

3. ДАШБОРД ПРОДУКТОВОЙ ЛИНЕЙКИ
📊 Анализ продуктовой линейки:
   • Категорий: 3
   • Подкатегорий: 11
   • Общая выручка по категориям: $28,895,568


✅ Дашборд продуктовой линейки создан

4. 👥 ДОПОЛНИТЕЛЬНЫЙ АНАЛИЗ КЛИЕНТОВ
❌ Пропущен дашборд анализа клиентов: отсутствуют YearlyIncome или Age

📊 ИТОГ СОЗДАНИЯ ДАШБОРДОВ:
   • Создано дашбордов: 4
   • Ключевые метрики: 29,307,837 $ выручки
   • Охвачено клиентов: 17,918
   • Обработано заказов: 26,683
✅ ВСЕ ДАШБОРДЫ УСПЕШНО СОЗДАНЫ И ОТОБРАЖЕНЫ!


In [14]:
# 💡 ИЗВЛЕЧЕНИЕ КЛЮЧЕВЫХ ИНСАЙТОВ
print("💡 КЛЮЧЕВЫЕ ИНСАЙТЫ")
print("=" * 50)

# 1. Общие метрики бизнеса
print("📈 ОБЩИЕ МЕТРИКИ БИЗНЕСА:")
total_revenue = df['SalesAmount'].sum()
total_customers = df['CustomerKey'].nunique()
total_products = df['ProductKey'].nunique()
avg_order_value = df.groupby('SalesOrderNumber')['SalesAmount'].sum().mean()

print(f"1. 💰 Общая выручка: ${total_revenue:,.0f}")
print(f"2. 👥 Количество клиентов: {total_customers}")
print(f"3. 📦 Количество продуктов: {total_products}")
print(f"4. 📊 Средний чек: ${avg_order_value:,.0f}")

# 2. Анализ клиентской базы
print("\n🎯 АНАЛИЗ КЛИЕНТСКОЙ БАЗЫ:")
customer_value = df.groupby('CustomerKey')['SalesAmount'].sum()
if len(customer_value) > 0:
    top_20_percent = int(len(customer_value) * 0.2)
    top_20_customers = customer_value.nlargest(top_20_percent).sum()
    percent_top_20 = (top_20_customers / total_revenue) * 100
    
    print(f"5. 🎯 Топ-20% клиентов приносят {percent_top_20:.1f}% выручки")
    print(f"   • Выручка от топ-20%: ${top_20_customers:,.0f}")
    print(f"   • Средняя выручка на клиента: ${customer_value.mean():,.0f}")
else:
    print("5. ❌ Недостаточно данных для анализа клиентской базы")

# 3. Анализ регионов
print("\n🌍 АНАЛИЗ РЕГИОНОВ:")
if 'Region' in df.columns:
    region_sales = df.groupby('Region')['SalesAmount'].sum().sort_values(ascending=False)
    best_region = region_sales.idxmax()
    best_region_revenue = region_sales.max()
    worst_region = region_sales.idxmin()
    worst_region_revenue = region_sales.min()
    
    print(f"6. 🌍 Лучший регион: {best_region} (${best_region_revenue:,.0f})")
    print(f"7. 📍 Худший регион: {worst_region} (${worst_region_revenue:,.0f})")
    print(f"   • Всего регионов: {len(region_sales)}")
    print(f"   • Разница между лучшим и худшим: {((best_region_revenue/worst_region_revenue)-1)*100:.1f}%")
else:
    print("6. ❌ Отсутствуют данные по регионам")

# 4. Анализ продуктов
print("\n📦 АНАЛИЗ ПРОДУКТОВ:")
if 'Category' in df.columns:
    category_sales = df.groupby('Category')['SalesAmount'].sum().sort_values(ascending=False)
    best_category = category_sales.idxmax()
    best_category_revenue = category_sales.max()
    
    print(f"8. 📦 Лучшая категория: {best_category} (${best_category_revenue:,.0f})")
    print(f"   • Доля лучшей категории: {(best_category_revenue/total_revenue)*100:.1f}%")
    print(f"   • Всего категорий: {len(category_sales)}")
else:
    print("8. ❌ Отсутствуют данные по категориям продуктов")

# 5. Временной анализ
print("\n📅 ВРЕМЕННОЙ АНАЛИЗ:")
if 'OrderDate' in df.columns:
    monthly_revenue = df.groupby(df['OrderMonth'])['SalesAmount'].sum()
    if len(monthly_revenue) > 0:
        best_month = monthly_revenue.idxmax()
        worst_month = monthly_revenue.idxmin()
        
        month_names = {
            1: 'Январь', 2: 'Февраль', 3: 'Март', 4: 'Апрель',
            5: 'Май', 6: 'Июнь', 7: 'Июль', 8: 'Август',
            9: 'Сентябрь', 10: 'Октябрь', 11: 'Ноябрь', 12: 'Декабрь'
        }
        
        print(f"9. 📈 Лучший месяц: {month_names.get(best_month, best_month)} (${monthly_revenue[best_month]:,.0f})")
        print(f"10. 📉 Худший месяц: {month_names.get(worst_month, worst_month)} (${monthly_revenue[worst_month]:,.0f})")
        print(f"    • Сезонность: разница {((monthly_revenue[best_month]/monthly_revenue[worst_month])-1)*100:.1f}%")
    else:
        print("9. ❌ Недостаточно данных для временного анализа")
else:
    print("9. ❌ Отсутствуют данные по датам заказов")

# 6. Анализ прибыльности
print("\n💵 АНАЛИЗ ПРИБЫЛЬНОСТИ:")
if 'Profit' in df.columns:
    total_profit = df['Profit'].sum()
    avg_profit_margin = (total_profit / total_revenue) * 100
    profitable_orders = len(df[df['Profit'] > 0])
    total_orders_count = len(df)
    profitability_rate = (profitable_orders / total_orders_count) * 100
    
    print(f"11. 💵 Общая прибыль: ${total_profit:,.0f}")
    print(f"12. 📊 Средняя маржа прибыли: {avg_profit_margin:.1f}%")
    print(f"    • Прибыльных заказов: {profitability_rate:.1f}% ({profitable_orders}/{total_orders_count})")
else:
    print("11. ❌ Отсутствуют данные по прибыли")

# 7. Дополнительные инсайты
print("\n🔍 ДОПОЛНИТЕЛЬНЫЕ ИНСАЙТЫ:")

# Анализ доставки
if 'DeliveryDays' in df.columns:
    avg_delivery = df['DeliveryDays'].mean()
    print(f"13. 🚚 Среднее время доставки: {avg_delivery:.1f} дней")

# Анализ демографии
if 'Age' in df.columns:
    avg_age = df['Age'].mean()
    print(f"14. 👤 Средний возраст клиентов: {avg_age:.1f} лет")

if 'YearlyIncome' in df.columns:
    avg_income = df['YearlyIncome'].mean()
    print(f"15. 💰 Средний годовой доход: ${avg_income:,.0f}")

print(f"\n📊 ИТОГО ИНСАЙТОВ:")
print(f"   • Основные инсайты: 12+")
print(f"   • Общая выручка: ${total_revenue:,.0f}")
print(f"   • Охвачено клиентов: {total_customers:,}")
print(f"   • Проанализировано продуктов: {total_products:,}")
print("✅ АНАЛИЗ ИНСАЙТОВ ЗАВЕРШЕН!")

💡 КЛЮЧЕВЫЕ ИНСАЙТЫ
📈 ОБЩИЕ МЕТРИКИ БИЗНЕСА:
1. 💰 Общая выручка: $29,307,837
2. 👥 Количество клиентов: 17918
3. 📦 Количество продуктов: 158
4. 📊 Средний чек: $1,098

🎯 АНАЛИЗ КЛИЕНТСКОЙ БАЗЫ:
5. 🎯 Топ-20% клиентов приносят 65.2% выручки
   • Выручка от топ-20%: $19,117,535
   • Средняя выручка на клиента: $1,636

🌍 АНАЛИЗ РЕГИОНОВ:
6. 🌍 Лучший регион: Australia ($9,051,766)
7. 📍 Худший регион: Central ($3,001)
   • Всего регионов: 10
   • Разница между лучшим и худшим: 301542.1%

📦 АНАЛИЗ ПРОДУКТОВ:
8. 📦 Лучшая категория: Bikes ($28,318,145)
   • Доля лучшей категории: 96.6%
   • Всего категорий: 3

📅 ВРЕМЕННОЙ АНАЛИЗ:
9. 📈 Лучший месяц: Декабрь ($3,180,924)
10. 📉 Худший месяц: Март ($1,834,668)
    • Сезонность: разница 73.4%

💵 АНАЛИЗ ПРИБЫЛЬНОСТИ:
11. 💵 Общая прибыль: $12,052,518
12. 📊 Средняя маржа прибыли: 41.1%
    • Прибыльных заказов: 100.0% (58189/58189)

🔍 ДОПОЛНИТЕЛЬНЫЕ ИНСАЙТЫ:
15. 💰 Средний годовой доход: $59,770

📊 ИТОГО ИНСАЙТОВ:
   • Основные инсайты: 12+
   • Общая выру